In [187]:
#Import necessary libraries
import os
from dotenv import load_dotenv
import json
import requests
from typing import Dict, List

load_dotenv()

#Constants
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

#Main API routes
SPOTIFY_API_URL_PREFIX = "https://api.spotify.com/v1"

In [188]:
def get_token(client_id:str=SPOTIFY_CLIENT_ID, client_secret:str=SPOTIFY_CLIENT_SECRET):
    """
    Retrieves a Spotify access token.

    Params:
        client_id (str): The Spotify Client ID.
        client_secret (str): The Spotify Client Secret.

    Returns:
        response_json (Dict(str, str, int)): A dictionary containing:
            - access_token (str): The Spotify access token.
            - token_type (str): The type of authorization token (e.g., "Bearer").
            - expires_in (int): The token's expiration time in seconds (typically 3600).
    """
    URL_TOKEN = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type":"application/x-www-form-urlencoded"
    }
    payload ={
        "grant_type":"client_credentials",
        "client_id":client_id,
        "client_secret":client_secret
    }
    try:
        response = requests.post(url=URL_TOKEN, headers=headers, data=payload)
        response_json = json.loads(response.content)
        return response_json
    except Exception as err:
        print(err)
        return {}

def get_auth_header(token: Dict[str,any]) -> Dict[str, str]:
    """
    Retrieves Spotify authorization header
    
    Params:
        token (Dict[str,any]) : token dictionary from get_token()
        
    Returns:
        authorization_header (Dict(str,str)): Bearer authorization header

    """
    authorization_header = {"Authorization": f"Bearer {token['access_token']}"}
    return authorization_header   

## Extract

In [229]:

def get_tracks_by_genre(genre:str, token:Dict[str,any],offset:int=0,limit:int=20):
    """
    Fetch tracks by genre from Spotify
    
    Params:
        genre (str): The genre to search for
        token (Dict[str,any]): Spotify authentication token
        offset (int, optiional): Pagination offset, Default to 0
        limit (int, optional): Number of tracks to fetch per request. Defaults to 20
    
    Returns:
        tracks (Dict[str,any]): A response detail from Spotify API
    
    """
    ENDPOINT = f"{SPOTIFY_API_URL_PREFIX}/search?q=genre:{genre}&type=track&offset={offset}&limit={limit}"
    headers = get_auth_header(token=token)
    response = requests.get(url=ENDPOINT, headers=headers)
    tracks = response.json()['tracks']
    return tracks

def get_track_details_from_tracks(tracks:Dict[str,any]):
    """
    Retrieves a list of track details 
    
    Params:
        tracks (Dict[str,any]): tracks from get_tracks_by_genre
    
    Returns:
        track_details (List[Dict[str,any]]): A list of dictionaries of new tracks details, containing:
            - track_id (str): Spotify ID for the track
            - track_name (str): Name of this track
            - track_url (str): Spotify URL for the object
            - duration_ms (int): The track length in milliseconds
            - artist_ids (List[str]): List of Spotify ID for the artists
            - album_id (str): Spotify ID for the album
            - release_date (str): The date the album was first released
            - is_single (Boolean):Check if this is a single or part of the album
            - explicit (Boolean): Whether or not the track has explicit lyrics
    """
    track_details = []
    for item in tracks['items']:
        track_artist_ids =[artist['id'] for artist in item['artists']]
        track_detail = {
            "track_id":item["id"],
            "track_name":item['name'],
            "track_url":item['external_urls']['spotify'],
            "duration_ms":item['duration_ms'],
            'artist_ids':track_artist_ids,
            "album_id":item['album']['id'],
            'release_date':item['album']['release_date'],
            "is_single":True if item['album']['album_type']=='single' else False,
            'explicit': item['explicit']
        }
        track_details.append(track_detail)
        
    return track_details
    
def get_album_details_from_tracks(tracks:Dict[str,any]):
    """
    Retrieves a list of album details from tracks
    
    Params:
        tracks (Dict[str,any]): tracks from get_tracks_by_genre
    
    Returns:
        album_details (List[Dict[str,any]]): A list of dictionaries of new album details, containing:
            - album_id (str): Spotify ID for the album
            - album_name (str): The name of the album. In case of an album takedown, the value may be an empty string
            - release_date (str): The date the album was first released
            - artist_ids (List[str]): List of Spotify ID for the artists
            - total_tracks (int): number of tracks in the album
            - image_640_url (str): URL for the album cover size 640x640
            _ image_300_url (str): URL for the album cover size 300x300
            _ image_64_url (str): URL for the album cover size 64x64
    """
    album_details = []
    for item in tracks['items']:
        album = item['album']
        album_artist_ids = [artist['id'] for artist in album['artists']]
        album_detail= {
            "album_id":album['id'],
            "album_name":album['name'],
            "release_date":album['release_date'],
            'artist_ids':album_artist_ids,
            'album_url':album['external_urls']['spotify'],
            'total_tracks':album['total_tracks'],
            "type":album['type'],
            "image_640_url":album['images'][0]["url"],
            "image_300_url":album['images'][1]["url"],
            'image_64_url':album['images'][2]['url']
        }
        album_details.append(album_detail)
        
    return album_details

def get_artist_track_dict_from_tracks(tracks:Dict[str,any]):
    """
    Retrieves artist who procduces the track
    
    Params:
        tracks (Dict[str,any]): tracks from get_tracks_by_genre
    
    Returns:
        artist_track_dict (Dict[str,List[str]]): A dictionary containing artist_id own a list of track
    """
    artist_track_dict:Dict[str,List[str]] = {}
    for item in tracks['items']:
        track_id = item['id']
        for artist in item['artists']:
            artist_id = artist['id']
            if artist_track_dict.get(artist_id) is None:
                artist_track_dict[artist_id] = []
            artist_track_dict[artist_id].append(track_id)
    return artist_track_dict
        
def get_artist_album_dict_from_tracks(tracks:Dict[str,any]):
    """
    Retrieves artist who procduces the album
    
    Params:
        tracks (Dict[str,any]): tracks from get_tracks_by_genre
    
    Returns:
        artist_album_dict (Dict[str,List[str]]): A dictionary containing artist_id own a list of album
    """
    
    artist_album_dict:Dict[str,List[str]] = {}
    for item in tracks['items']:
        album = item['album']
        album_id = album['id']
        for artist in item['artists']:
            artist_id = artist['id']
            if artist_album_dict.get(artist_id) is None:
                artist_album_dict[artist_id] = []
            artist_album_dict[artist_id].append(album_id)
    return artist_album_dict
        
def get_artist_detail(artist_id:str, token:Dict[str,any]):
    '''
    Retrieve artist information from Spotify
    
    Params:
        artist_id (str): Spotify Id of the artist
    
    Returns:
        artist_detail (Dict[str,str]): A dictionary of artist detail, containing:
            - artist_id (str): Spotify ID for the artist
            - name (str): The name of the artist
            - url (str): Spotify URL for the object
            - genres (List[str]): A list of the genres thr artist is associated with. If not yet classified, the array is empty
            - image_640_url (str): URL for the artist cover size 640x640
            - image_320_url (str): URL for the artist cover size 320x320
            - image_160_url (str): URL for the artist cover size 160x160
        
    '''
    ENDPOINT = f"{SPOTIFY_API_URL_PREFIX}/artists/{artist_id}"
    headers = get_auth_header(token=token)
    response = requests.get(url=ENDPOINT, headers=headers)
    artist_doc = response.json()
    artist_detail = {
        "artist_id": artist_doc['id'],
        "name": artist_doc['name'],
        "url":artist_doc['external_urls']['spotify'],
        'genres':artist_doc['genres'],
        'image_640_url':artist_doc['images'][0]['url'],
        'image_320_url':artist_doc['images'][1]['url'],
        'image_160_url':artist_doc['images'][2]['url']
    }
    return artist_detail
    
    
    

## Transform

In [230]:
import pandas as pd

token = get_token()
dubstep_tracks = get_tracks_by_genre(genre='dubstep', token=token, limit=50,offset=100)

dubstep_track_details = get_track_details_from_tracks(dubstep_tracks)
dubstep_album_details = get_album_details_from_tracks(dubstep_tracks)
dubstep_artist_tracks = get_artist_track_dict_from_tracks(dubstep_tracks)
dubstep_artist_albums = get_artist_album_dict_from_tracks(dubstep_tracks)
artist_ids = set(list(dubstep_artist_tracks.keys()) + list(dubstep_artist_albums.keys()))
artist_details = [get_artist_detail(artist_id, token=token) for artist_id in artist_ids]


#DataFrame Create
dubstep_track_df = pd.DataFrame(dubstep_track_details).drop(columns=["artist_ids"])
dubstep_album_df = pd.DataFrame(dubstep_album_details).drop(columns=['artist_ids'])
dubstep_artist_df = pd.DataFrame(artist_details)
artist_tracks_df = pd.DataFrame({"artist_id": list(dubstep_artist_tracks.keys()), "track_id": dubstep_artist_tracks.values()}).explode("track_id")
artist_albums_df = pd.DataFrame({"artist_id": list(dubstep_artist_albums.keys()), "album_id": dubstep_artist_albums.values()}).explode("album_id")





In [234]:
def get_genre_list_from_artist(artist_details):
    genre_list = []
    for artist_detail in artist_details:
        genre_list.extend(artist_detail['genres'])
    return list(set(genre_list))


artist_genres_dict = {artist_detail['artist_id']:artist_detail['genres'] for artist_detail in artist_details}
dubstep_track_df


,track_id,track_name,track_url,duration_ms,album_id,release_date,is_single,explicit
0,4uZuF80Yw8KgW6d7wtmRlw,Alive - Trivecta Remix,https://open.spotify.com/track/4uZuF80Yw8KgW6d...,247741,3MBXaGZS6lr8DhZxjkYTJi,2019-08-30,False,False
1,1AcoISuZA2tq5uFOwRA7yu,You Don't Even Know Me,https://open.spotify.com/track/1AcoISuZA2tq5uF...,226000,6OWcBFdfI15aown78xchVh,2018-09-20,True,False
2,5yHvGn616eJSHay3d2SuyR,Better Than Heaven,https://open.spotify.com/track/5yHvGn616eJSHay...,203200,1418YAtqfNiXy6SqnkK65z,2020-05-21,True,False
3,1il2jHl3AdodyqQb7fQmlZ,Luv Me a Little,https://open.spotify.com/track/1il2jHl3AdodyqQ...,224615,0ValflNlWce21wm5PlwV60,2023-04-28,False,False
4,24KFdeWtel3jLwZ399zwq2,ACID,https://open.spotify.com/track/24KFdeWtel3jLwZ...,178285,3hObmnjaub2zwiSM73tQu9,2024-04-17,True,False
5,0bey95Gcplo3PsvEubZjv1,Bloom,https://open.spotify.com/track/0bey95Gcplo3Psv...,240000,5OhJBPKs7owuzzBxKIc01G,2019-03-22,False,False
6,1UtRBq2n0IlQNA1WI3axWX,Deep End,https://open.spotify.com/track/1UtRBq2n0IlQNA1...,208421,2Mzaihfp9i7m6UO5HSRtHb,2021-11-05,False,False
7,6POZ7ocsOQQvJrfArz6KOD,Save Me (From Myself) (feat. Kyle Hume),https://open.spotify.com/track/6POZ7ocsOQQvJrf...,218666,47VbVxpe5CtPMOFCpzliMr,2023-04-07,True,False
8,1mKxlggsbt21Ku6oD5wuX0,Leaving,https://open.spotify.com/track/1mKxlggsbt21Ku6...,234857,7A9I9BrNylVCqXzgN3Y6qi,2017-09-21,False,False
9,5Z5fBGwWP4x0mYCmchLpc1,Vessel,https://open.spotify.com/track/5Z5fBGwWP4x0mYC...,155200,5f9SHWzhvxa82Tm2icAptO,2023-03-31,True,False
